# Analyse de la méthode SLIC

Ce notebook démontre l'utilisation complète de la méthode SLIC implémentée.

## Table des matières
1. Chargement des données
2. Application de SLIC avec paramètres par défaut
3. Calcul des métriques de qualité
4. Comparaison de différents paramètres
5. Analyse détaillée des superpixels
6. Évaluation avec Ground Truth
7. Analyse de la compacité
8. Sauvegarde des résultats

In [ ]:
# Imports
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

from src.methods.slic.slic_original import SLIC
from src.preprocessing.image_loader import ImageLoader
from src.evaluation.metrics import compute_all_metrics, format_metrics, compute_metrics_multiple_gt
from src.evaluation.visualize import (
    visualize_segmentation, 
    compare_segmentations,
    visualize_superpixel_sizes,
    create_summary_figure
)
from src.methods.slic.utils import (
    compare_parameters,
    plot_compactness_distribution,
    print_summary
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

## 1. Chargement d'une image

Nous allons charger une image du dataset BSDS500 ou créer une image synthétique.

In [ ]:
# Option 1: Charger une image du dataset BSDS500
try:
    loader = ImageLoader(data_dir='../data')
    images, paths = loader.load_bsds500_images(split='test', max_images=1)
    
    if images:
        image = images[0]
        image_path = paths[0]
        print(f"Image chargée: {image_path}")
    else:
        raise FileNotFoundError("Aucune image trouvée")
        
except Exception as e:
    print(f"Impossible de charger BSDS500: {e}")
    print("Création d'une image synthétique...")
    
    # Option 2: Créer une image synthétique
    image = np.zeros((300, 400, 3), dtype=np.uint8)
    image[:, :] = [100, 150, 200]  # Fond bleu
    
    # Cercle rouge
    y, x = np.ogrid[:300, :400]
    mask_circle = (x - 150)**2 + (y - 150)**2 <= 50**2
    image[mask_circle] = [220, 50, 50]
    
    # Rectangle vert
    image[50:100, 250:350] = [50, 200, 50]
    
    # Rectangle jaune
    image[200:250, 250:350] = [230, 230, 50]
    
    # Ajouter du bruit
    noise = np.random.randint(-20, 20, image.shape, dtype=np.int16)
    image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    image_path = "synthetic"

print(f"Taille de l'image: {image.shape}")

# Afficher l'image
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title('Image originale', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 2. Application de SLIC avec paramètres par défaut

Appliquons SLIC avec les paramètres standards:
- n_segments = 200
- compactness = 10
- max_iter = 25

In [ ]:
# Créer et appliquer SLIC
slic = SLIC(n_segments=200, compactness=10, max_iter=25)
labels = slic.fit(image)

print(f"\nNombre de superpixels générés: {len(np.unique(labels))}")

# Obtenir les statistiques
stats = slic.get_statistics()
print(f"Taille moyenne: {stats['mean_size']:.1f} pixels")
print(f"Espacement de grille S: {stats['grid_spacing_S']}")

# Visualiser
fig = visualize_segmentation(image, labels, 
                             title="SLIC - Paramètres par défaut (n=200, m=10)")
plt.show()

## 3. Métriques de qualité

Calculons les métriques de qualité de la segmentation (sans ground truth).

In [ ]:
# Calculer les métriques
metrics = compute_all_metrics(labels)

# Afficher de manière formatée
print(format_metrics(metrics))

# Afficher également sous forme de dictionnaire
print("\nMétriques détaillées:")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 4. Comparaison de différents paramètres

### 4.1 Effet du nombre de superpixels (n_segments)

In [ ]:
# Tester différentes valeurs de n_segments
n_segments_list = [50, 100, 200, 400]
compactness_list = [10]

print("Comparaison du nombre de superpixels...")
fig, results = compare_parameters(image, n_segments_list, compactness_list, max_iter=20)
plt.suptitle('Effet du nombre de superpixels (n_segments)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Afficher les métriques pour chaque configuration
print("\nMétriques par configuration:")
for key, lbls in results.items():
    mets = compute_all_metrics(lbls)
    print(f"\n{key}:")
    print(f"  N réel: {mets['n_superpixels']}")
    print(f"  Compacité: {mets['compactness']:.4f}")
    print(f"  Régularité: {mets['regularity']:.4f}")

### 4.2 Effet du paramètre de compacité (m)

In [ ]:
# Tester différentes valeurs de compactness
n_segments_list = [200]
compactness_list = [1, 5, 10, 20, 40]

print("Comparaison de la compacité...")
fig, results = compare_parameters(image, n_segments_list, compactness_list, max_iter=20)
plt.suptitle('Effet du paramètre de compacité (m)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Afficher les métriques
print("\nMétriques par niveau de compacité:")
for key, lbls in results.items():
    mets = compute_all_metrics(lbls)
    print(f"\n{key}:")
    print(f"  Compacité mesurée: {mets['compactness']:.4f}")
    print(f"  Régularité: {mets['regularity']:.4f}")

## 5. Analyse détaillée des superpixels

### 5.1 Distribution des tailles

In [ ]:
# Distribution des tailles
fig = visualize_superpixel_sizes(labels)
plt.show()

# Afficher un résumé textuel
print_summary(image, labels)

### 5.2 Analyse de la compacité

In [ ]:
from src.methods.slic.utils import analyze_compactness

# Analyser la compacité
compactness_scores, mean_comp = analyze_compactness(labels)

# Visualiser la distribution
fig = plot_compactness_distribution(labels, bins=30)
plt.show()

print(f"\nStatistiques de compacité:")
print(f"  Moyenne: {mean_comp:.4f}")
print(f"  Médiane: {np.median(compactness_scores):.4f}")
print(f"  Écart-type: {np.std(compactness_scores):.4f}")
print(f"  Min/Max: {np.min(compactness_scores):.4f} / {np.max(compactness_scores):.4f}")

## 6. Comparaison avec différentes configurations

Générons et comparons plusieurs segmentations.

In [ ]:
# Générer plusieurs segmentations
labels_dict = {}

configs = [
    {'n_segments': 100, 'compactness': 10, 'name': 'SLIC-100'},
    {'n_segments': 200, 'compactness': 10, 'name': 'SLIC-200'},
    {'n_segments': 400, 'compactness': 10, 'name': 'SLIC-400'},
]

print("Génération des segmentations...")
for config in configs:
    print(f"  {config['name']}...")
    slic_temp = SLIC(n_segments=config['n_segments'], 
                     compactness=config['compactness'],
                     max_iter=20)
    labels_dict[config['name']] = slic_temp.fit(image)

# Comparer visuellement
fig = compare_segmentations(image, labels_dict)
plt.show()

# Comparer les métriques
from src.evaluation.visualize import plot_metrics_comparison
from src.evaluation.metrics import compare_segmentations_metrics

comparison = compare_segmentations_metrics(labels_dict)
fig = plot_metrics_comparison(comparison)
plt.show()

## 7. Évaluation avec Ground Truth (si disponible)

Si nous avons accès au ground truth du BSDS500, nous pouvons calculer les métriques complètes.

In [ ]:
# Charger le ground truth
try:
    if image_path != "synthetic":
        image_name = Path(image_path).stem
        loader = ImageLoader(data_dir='../data')
        gt_data = loader.load_bsds500_groundtruth(split='test', image_name=image_name)
        
        if gt_data and len(gt_data) > 0:
            ground_truths = gt_data[0]
            print(f"Ground truth chargé: {len(ground_truths)} segmentations\n")
            
            # Calculer les métriques avec GT
            metrics_with_gt = compute_metrics_multiple_gt(labels, ground_truths)
            print(format_metrics(metrics_with_gt))
            
            # Visualiser comparaison
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Image originale
            axes[0].imshow(image)
            axes[0].set_title('Image originale', fontweight='bold')
            axes[0].axis('off')
            
            # Ground Truth
            from skimage import segmentation as seg
            img_norm = image / 255.0 if image.max() > 1 else image
            marked_gt = seg.mark_boundaries(img_norm, ground_truths[0], 
                                           color=(1, 0, 0), mode='thick')
            axes[1].imshow(marked_gt)
            axes[1].set_title('Ground Truth', fontweight='bold', color='red')
            axes[1].axis('off')
            
            # SLIC
            marked_slic = seg.mark_boundaries(img_norm, labels, 
                                             color=(1, 1, 0), mode='thick')
            axes[2].imshow(marked_slic)
            axes[2].set_title('SLIC', fontweight='bold', color='orange')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
        else:
            print("Pas de ground truth disponible pour cette image")
    else:
        print("Image synthétique - pas de ground truth")
        
except Exception as e:
    print(f"Erreur lors du chargement du ground truth: {e}")
    print("Utilisation uniquement des métriques sans GT")

## 8. Figure récapitulative complète

In [ ]:
# Créer une figure récapitulative
fig = create_summary_figure(image, labels, metrics, 
                            title="Résumé de la segmentation SLIC")
plt.show()

## 9. Sauvegarde des résultats

In [ ]:
from src.methods.slic.utils import save_segmentation
from src.evaluation.metrics import save_metrics_to_csv

# Créer le dossier de résultats
results_dir = Path('../results/slic/notebook_analysis')
results_dir.mkdir(parents=True, exist_ok=True)

# Sauvegarder les labels
labels_path = results_dir / 'labels.npy'
save_segmentation(labels, labels_path)

# Sauvegarder les métriques
metrics_path = results_dir / 'metrics.csv'
save_metrics_to_csv(metrics, metrics_path)

# Sauvegarder la figure principale
fig = visualize_segmentation(image, labels)
fig.savefig(results_dir / 'segmentation.png', dpi=150, bbox_inches='tight')
plt.close(fig)

print(f"\nRésultats sauvegardés dans: {results_dir}")

## 10. Expérimentation libre

Cellule de test.

In [ ]:
# Exemple: SLIC avec paramètres extrêmes
slic_extreme = SLIC(n_segments=1000, compactness=1, max_iter=60)
labels_extreme = slic_extreme.fit(image)

fig = visualize_segmentation(image, labels_extreme,
                             title="SLIC avec paramètres extrêmes")
plt.show()